# Text Mining
Classification de Textes
Dans ce projet de Text Mining, nous allons classer des articles de presse et plus précisement essayer de détecter s'ils parlent de business ou non. C'est une tâche de **classification supervisée binaire**.

La base est disponnible sur le site de kaggle, c'est une base extraite de BBC News Archive : https://www.kaggle.com/datasets/hgultekin/bbcnewsarchive

# Import des librairies utiles au module


In [4]:
# Pour le data management 
import pandas as pd
import numpy as np

# Pour le pré processing
from unidecode import unidecode
import re
from nltk.stem import SnowballStemmer

# Les bigrammes
from collections import Counter
from nltk.util import ngrams

# Pour la vectorisation
from sklearn.feature_extraction.text import TfidfVectorizer 

# Pour la modélisation
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score

from sklearn.naive_bayes import GaussianNB

In [ ]:
1. Découverte et exploration des données¶
1.1. Import de la base
La base contient 2225 articles et 4 variables :

category : Le type d'article rédigé. il servira à la création de la cible à partir de cette variable.
content : Le contenu de l'article.
nom de fichier : Nom de fichier original qui contient le contenu de la nouvelle correspondante.
title : Le titre de l'article


In [11]:
##df = pd.read_csv('bbc-news-data.csv')
df = pd.read_csv("bbc-news-data.csv", delimiter='\t')

print(f"la table fait {df.shape[0]} lignes et {df.shape[1]} colonnes" )

df.head()

la table fait 2225 lignes et 4 colonnes


,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


Vérifions le type de données sur laquelle on travaille et si la base contient des valeurs manquantes.

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   filename  2225 non-null   object
 2   title     2225 non-null   object
 3   content   2225 non-null   object
dtypes: object(4)
memory usage: 69.7+ KB


1.2. Rapide nettoyage de la base
On remarque que la base ne contient pas de données manquantes. Donc pas besoin de nettoyer la base. On peut passer cette étape

1.3. Rapide exploration des données¶

Répartition de la colonne cible et préparation
l'objectif ici est d'entrainer un classifieur capable de détecter si l'article est un article de business.

Cette information est contenue dans la variable category.

In [14]:
###Fréquence d'apparition de chaque mot de 'category' en pourcentage 
df.category.value_counts(normalize=True)


category
sport            0.229663
business         0.229213
politics         0.187416
tech             0.180225
entertainment    0.173483
Name: proportion, dtype: float64

Le taux de cible pour ce projet est d'environ 23% (business)

In [17]:
###Créer une colonne 'business' qui est égale à 1 si category est 'business' sinon 0
df['business'] = df['category'].apply(lambda category: 1 if category == 'business' else 0)
###Calculer la fréquence de 0 & 1 dans cette nouvelle colonne en pourcentage 
df['business'].value_counts(normalize=True)

business
0    0.770787
1    0.229213
Name: proportion, dtype: float64

Visualisation d'un exemple d'article de chaque type¶

Comme pour les données "plus classiques", lorsqu'on travaille sur des textes, il est important d'explorer les données.
Cela peut donner des idées de features, et permet d'éviter certains pièges, etc...

In [18]:
for category in df['category'].unique():

    print(f'####### Articles {category} #######')
    print(f"La base contient {df[df['category']==category].shape[0]} articles - Exemple du 1er article : ")
    print(f"Titre - {df[df['category']==category]['title'].tolist()[0]} \n\n{df[df['category']==category]['content'].tolist()[0]}")
    print('###################################')
    print('\n')

####### Articles business #######
La base contient 510 articles - Exemple du 1er article : 
Titre - Ad sales boost Time Warner profit 

 Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscr

Remarques :

De nombreux features peuvent être extrait des textes à des fins exploratoire ou de modélisation : les années, les noms prénoms, les unités, les heures, ...

In [20]:
df.head()

,category,filename,title,content,business
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,1
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,1
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,1
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,1
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,1


2. Nettoyage du texte¶
Nous appliquons les étapes du pré-processing ici:la normalisation du texte
2.1. Nettoyage et stemmatisation

In [22]:
# Définition de la liste de stop words considérés (celle de spacy + lettres)
stopWords = [
    'a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 
    'although', 'always', 'am', 'among', 'amongst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 
    'anywhere', 'are', 'around', 'as', 'at', 'b', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 
    'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'c', 
    'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'd', 'de', 'describe', 'detail', 'do', 'done', 'down', 
    'due', 'during', 'e', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 
    'every', 'everyone', 'everything', 'everywhere', 'except', 'f', 'few', 'fifteen', 'fify', 'fill', 'find', 'fire', 'first', 'five', 
    'for', 'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'g', 'get', 'give', 'go', 'h', 'had', 
    'has', 'hasnt', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 
    'himself', 'his', 'how', 'however', 'hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed', 'into', 'is', 'it', 'its', 'itself', 'j', 
    'k', 'keep', 'l', 'last', 'latter', 'latterly', 'least', 'less', 'ltd', 'm', 'made', 'many', 'may', 'me', 'meanwhile', 'might', 
    'mill', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'my', 'myself', 'n', 'name', 'namely', 'neither', 
    'never', 'nevertheless', 'next', 'nine', 'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'o', 'of', 
    'off', 'often', 'on', 'once', 'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 
    'over', 'own', 'p', 'part', 'per', 'perhaps', 'please', 'put', 'q', 'r', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed', 
    'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'so', 'some', 
    'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'system', 't', 'take', 'ten', 'than', 
    'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 
    'thereupon', 'these', 'they', 'thickv', 'thin', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 
    'thus', 'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'u', 'under', 'until', 'up', 'upon', 
    'us', 'very', 'via', 'was', 'we', 'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 
    'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 
    'whom', 'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', 
    'z', 'qu'
]
stopWords = [unidecode(sw) for sw in stopWords]

# Création du stemmer
stemmer = SnowballStemmer('english')

In [23]:
# Création d'une fonction pour supprimer les sw
def no_stop_word(string, stopWords):

    """
    Supprime les stop words d'un texte.

    Paramètres
    ----------

    string : chaine de caractère.

    stopWords : liste de mots à exclure. 
    
    ----------
    Sortie : string sans stopWords
    """
    
    string = ' '.join([word for word in string.split() if word not in stopWords])
    return string

# Création d'une fonction pour stemmatiser chaque mot d'un text 
def stemmatise_text(text, stemmer):

    """
    Stemmatise un texte : Ramène les mots d'un texte à leur racine (peut créer des mots qui n'existe pas).

    Paramètres
    ----------

    text : Chaine de caractères.

    stemmer : Stemmer de NLTK.
    
    ----------
    Sortie : string qui contient la forme stemmatisée des mots
    """
    
    string = " ".join([stemmer.stem(word) for word in text.split()])

    return string

In [26]:
def stem_cleaner(pandasSeries, stemmer, stopWords):
    
    print("#### Nettoyage en cours ####") # Mettre des print vous permet de comprendre où votre code rencontre des problèmes en cas de bug
    
    # confirmation que chaque article est bien de type str
    pandasSeries = pandasSeries.apply(lambda x : str(x))
    
    # Passage en minuscule
    print("... Passage en minuscule") 
    pandasSeries = pandasSeries.apply(lambda x : x.lower())
    
    # Suppression des accents
    #print("... Suppression des accents") 
    #pandasSeries = pandasSeries.apply(lambda x: unidecode(x))
    
    # Détection du champs année, c'est pour l'exemple, l'articles de business contient les années 
   
    # Changement de chaque année numérique en 'annee' en utilisant une regex
    print("... Détection du champs année") 
    pandasSeries = pandasSeries.apply(lambda x: re.sub(r"[0-9]{4}","annee",x))
    
    # Suppression des caractères spéciaux et numériques
    # Garder uniquement les lettres a-z en utilisant une regex
    print("... Suppression des caractères spéciaux et numériques") 
    pandasSeries = pandasSeries.apply(lambda x:re.sub(r"[^a-z]+"," ",x))
    
    # Suppression des stop words (appliquer la fonction no_stop_word créée ci-dessus)
    print("... Suppression des stop words") 
    pandasSeries = pandasSeries.apply(lambda x:no_stop_word(x, stopWords))
    
    # Stemmatisation (appliquer la fonction stemmatise_text créée ci-dessus)
    print("... Stemmatisation") 
    pandasSeries = pandasSeries.apply(lambda x:stemmatise_text(x, stemmer))
    
    print("#### Nettoyage OK! ####")

    return pandasSeries

In [27]:
%%time 
df['content_stem'] = stem_cleaner(df['content'], stemmer, stopWords)

df[['content', 'content_stem']].head()

#### Nettoyage en cours ####
... Passage en minuscule
... Détection du champs année
... Suppression des caractères spéciaux et numériques
... Suppression des stop words
... Stemmatisation
#### Nettoyage OK! ####
CPU times: total: 18.6 s
Wall time: 27.8 s


,content,content_stem
0,Quarterly profits at US media giant TimeWarne...,quarter profit media giant timewarn jump bn mo...
1,The dollar has hit its highest level against ...,dollar hit highest level euro month feder rese...
2,The owners of embattled Russian oil giant Yuk...,owner embattl russian oil giant yuko ask buyer...
3,British Airways has blamed high fuel prices f...,british airway blame high fuel price drop prof...
4,Shares in UK drinks and food firm Allied Dome...,share uk drink food firm alli domecq risen spe...


In [28]:
df[['content', 'content_stem']].tail()

,content,content_stem
2220,BT is introducing two initiatives to help bea...,bt introduc initi help beat rogu dialler scam ...
2221,Computer users across the world continue to i...,comput user world continu ignor secur warn spa...
2222,A new European directive could put software w...,new european direct softwar writer risk legal ...
2223,The man making sure US computer networks are ...,man make sure comput network safe secur resign...
2224,"Online role playing games are time-consuming,...",onlin role play game time consum enthral fligh...


4. Vectorisation et modélisation ¶
Nous utilisons ici un modèle simple de Machine Learning

4.1. Création du jeu de test et du jeu de train
Dans les modules de Machine Learning, lorsqu'on entraine un modèle, il faut monitorer ses performances et sa capacité à se généraliser sur un nouveau jeu de données. C'est le compromis entre optimisation et généralisation.

C'est pourquoi ici, nous créons :

un jeu d'entrainement : Pour entainer notre algorithme
un jeu de test : Pour tester la capacité de notre modèle à se généraliser
Nous n'en dirons pas plus car ce n'est pas l'objet du module de texte.

In [30]:
train, test = train_test_split(df, 
                               random_state=42,     # Permet de fixer la répartition de nos données dans le train et le test
                               test_size=0.2,       # Ici on reserve 20% de données pour le jeu de test
                               stratify=df["business"] # On s'assure d'avoir le même taux de cible dans le train et le test
                              )

for df_ in [train, test]:
    df_.reset_index(drop=True, inplace=True)

print(f"Notre jeu de train contient {train.shape[0]} observations.")
print(f"Notre jeu de test contient {test.shape[0]} observations.")

Notre jeu de train contient 1780 observations.
Notre jeu de test contient 445 observations.


In [31]:
train.head()

,category,filename,title,content,business,content_stem
0,politics,185.txt,Police probe BNP mosque leaflet,Police are investigating a British National P...,0,polic investig british nation parti leaflet po...
1,entertainment,373.txt,British stars denied major Oscars,British hopes of winning major Oscars were da...,0,british hope win major oscar dash uk star fail...
2,sport,055.txt,Holmes is hit by hamstring injury,Kelly Holmes has been forced out of this week...,0,kelli holm forc weekend european indoor athlet...
3,tech,221.txt,Halo 2 heralds traffic explosion,The growing popularity of online gaming could...,0,grow popular onlin game spell problem net serv...
4,entertainment,040.txt,Wine comedy up for six film gongs,"Sideways, a wine-tasting comedy starring Paul...",0,sideway wine tast comedi star paul giamatti in...


In [32]:
test.head()

,category,filename,title,content,business,content_stem
0,sport,019.txt,London hope over Chepkemei,London Marathon organisers are hoping that ba...,0,london marathon organis hope ban athlet susan ...
1,tech,224.txt,Disney backs Sony DVD technology,A next generation DVD technology backed by So...,0,generat dvd technolog back soni receiv major b...
2,tech,069.txt,'Friends fear' with lost mobiles,People are becoming so dependent on their mob...,0,peopl depend mobil phone concern lose phone me...
3,entertainment,294.txt,No UK premiere for Rings musical,The producers behind the Lord of the Rings mu...,0,produc lord ring music abandon plan premier lo...
4,tech,059.txt,Mobiles 'not media players yet',"Mobiles are not yet ready to be all-singing, ...",0,mobil readi sing danc multimedia devic replac ...


In [35]:
# Contrôle des taux de cible
print(f"Le taux de cible du train est de {round(train['business'].sum()/len(train), 3)}")
print(f"Le taux de cible du test est de {round(test['business'].sum()/len(test), 3)}")

Le taux de cible du train est de 0.229
Le taux de cible du test est de 0.229


4.2. Vectorisation
Nous faisons le choix d'utiliser le TfidfVectorizer de scikit-learn.
Un count vectorizer ou des plongements de mots pourrait être plus performant. Ca se test, tout comme le pré-processing !

In [36]:
tfidf = TfidfVectorizer(min_df=100,        # Réduit le nombre de features
                        ngram_range=(1, 3) # On inclu les unigrammes, les bigrammes et les trigrammes
                       ) 

tfidf.fit(train.loc[:,('content_stem')].values.astype('U'))

X_train_tfidf = tfidf.transform(train['content_stem'].values.astype('U')).toarray()
X_test_tfidf = tfidf.transform(test['content_stem'].values.astype('U')).toarray()
 
print(f"Le jeu de train contient {X_train_tfidf.shape[0]} obs et {X_train_tfidf.shape[1]} features") 

print(f"Le jeu de test contient {X_test_tfidf.shape[0]} obs et {X_test_tfidf.shape[1]} features") 

print('')

print('Le nombre de features correspond à la taille de votre vocabulaire !')

Le jeu de train contient 1780 obs et 568 features
Le jeu de test contient 445 obs et 568 features

Le nombre de features correspond à la taille de votre vocabulaire !


In [41]:
y_train = train['business']
y_test = test['business']

y_train.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    0
8    0
9    0
Name: business, dtype: int64

4.3. Modélisation¶
Dans cette partie nous ne rentrerons pas dans le détail, l'idée est simplement de montrer comment entrainer un classifieur capable de classer nos documents.

Ici nous faisons le choix d'utiliser un classifieur Bayesien Naif pour créer un modèle base line. Nous faisons ce choix, car il est très rapide à entrainer.

Nous pouvons challenger ce modèle en améliorant le pre processing et en affinant le choix de l'algorithme.

A savoir :
Le modèle Bayésien naïf, également connu sous le nom de classificateur naïf de Bayes, est un modèle de classification probabiliste basé sur le théorème de Bayes. Il suppose que les caractéristiques sont indépendantes les unes des autres conditionnellement à la classe cible. Bien que cette hypothèse d'indépendance soit souvent simpliste et irréaliste dans la pratique, le modèle Bayésien naïf reste simple et efficace dans de nombreuses situations.

Fonctionnement du modèle Bayésien naïf :
Collecte des données : Tout d'abord, nous collectons un ensemble de données d'entraînement qui contient des exemples étiquetés, où chaque exemple est décrit par un ensemble de caractéristiques et une étiquette de classe.

Calcul des probabilités a priori : Pour chaque classe possible, nous calculons la probabilité a priori de cette classe en comptant simplement le nombre d'exemples d'entraînement appartenant à cette classe et en le divisant par le nombre total d'exemples d'entraînement.

Calcul des probabilités conditionnelles : Pour chaque caractéristique et pour chaque valeur possible de cette caractéristique, nous calculons la probabilité conditionnelle de cette valeur sachant la classe. Cela peut être fait en comptant le nombre d'exemples d'entraînement dans chaque classe pour lesquels cette caractéristique prend la valeur donnée, puis en le divisant par le nombre total d'exemples d'entraînement dans cette classe.

Prédiction : Pour un nouvel exemple dont les caractéristiques sont connues mais dont la classe est inconnue, nous utilisons le théorème de Bayes pour calculer la probabilité conditionnelle de chaque classe donnée, les caractéristiques de cet exemple. La classe prédite est alors celle qui maximise cette probabilité conditionnelle.

Évaluation du modèle : Une fois que le modèle a été entraîné et que des prédictions ont été faites sur un ensemble de données de test distinct, nous évaluons ses performances en comparant les étiquettes prédites avec les étiquettes réelles des exemples de test.

Le principal avantage du modèle Bayésien naïf est sa simplicité et sa rapidité, ce qui le rend efficace pour de nombreux problèmes de classification, en particulier lorsque les données sont de haute dimensionnalité. Cependant, sa principale faiblesse réside dans son hypothèse d'indépendance conditionnelle, qui peut être trop simpliste pour certains ensembles de données où les caractéristiques sont fortement corrélées.

In [42]:
%%time
gssnNB = GaussianNB()
gssnNB.fit(X_train_tfidf, y_train.values)

y_predicted_train = gssnNB.predict(X_train_tfidf)
y_predicted_test = gssnNB.predict(X_test_tfidf)
    
print('Train performances : {} obs ... f1 score = {:0.4f} ... precision = {:0.4f} ... recall = {:0.4f}'.format(sum(y_train.values), 
                                                                                                                   f1_score(y_train.values,y_predicted_train),
                                                                                                                   precision_score(y_train.values, y_predicted_train),
                                                                                                                   recall_score(y_train.values, y_predicted_train)))

print('Test performances : {} obs ... f1 score = {:0.4f} ... precision = {:0.4f} ... recall = {:0.4f}'.format(sum(y_test.values), 
                                                                                                                  f1_score(y_test.values, y_predicted_test),
                                                                                                                  precision_score(y_test.values, y_predicted_test),
                                                                                                                  recall_score(y_test.values, y_predicted_test)))

Train performances : 408 obs ... f1 score = 0.9471 ... precision = 0.9292 ... recall = 0.9657
Test performances : 102 obs ... f1 score = 0.9372 ... precision = 0.9238 ... recall = 0.9510
CPU times: total: 62.5 ms
Wall time: 129 ms


On constate que le modèle Bayésien naïf est performant pour prédire si on a un article de business ou pas, car les mesures de performance sont proches de 1 et sont sensiblement les mêmes pour Train performances, que pour Test performances.

Le F1-score est une mesure de performance couramment utilisée pour évaluer les modèles de classification, en particulier lorsque les classes sont déséquilibrées. Il est calculé à partir des métriques de rappel (recall) et de précision (precision) et est une moyenne harmonique de ces deux mesures.

Précision (Precision)
La précision est le nombre de vrais positifs (TP) divisé par le nombre total de prédictions positives (TP + FP). En d'autres termes, c'est la proportion des prédictions positives qui sont correctes.

𝑃𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛=𝑇𝑃𝑇𝑃+𝐹𝑃
 
Rappel (Recall)
Le rappel est le nombre de vrais positifs (TP) divisé par le nombre total d'instances qui sont réellement positives (TP + FN). En d'autres termes, c'est la proportion des instances positives qui ont été correctement prédites.

𝑅𝑒𝑐𝑎𝑙𝑙=𝑇𝑃𝑇𝑃+𝐹𝑁
 
F1-score
Le F1-score est la moyenne harmonique de la précision et du rappel. Il est calculé comme suit :

𝐹1=2∗𝑃𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛∗𝑅𝑒𝑐𝑎𝑙𝑙𝑃𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛+𝑅𝑒𝑐𝑎𝑙𝑙
 
Le F1-score est utile lorsque vous voulez avoir un équilibre entre la précision et le rappel. Il atteint sa meilleure valeur à 1 et sa pire valeur à 0. Une haute valeur de F1 indique à la fois une précision et un rappel élevés, ce qui signifie que le modèle a bien prédit à la fois les vrais positifs et les vrais négatifs. C'est particulièrement important lorsque les classes sont déséquilibrées, car une précision élevée peut être obtenue simplement en prédisant toujours la classe majoritaire, mais cela entraînera généralement un rappel très faible pour la classe minoritaire.

En résumé, le F1-score est une mesure utile pour évaluer la performance d'un modèle de classification, en particulier dans les cas où les classes sont déséquilibrées et où vous voulez un équilibre entre la précision et le rappel.